## Задание 2.2
### Порождение подсказок по словарю
#### Бор

In [1]:
class Trie:
    def __init__(self):
        self.children = {}
        self.word = None
    
    
    def add(self, word):
        pos = self
        for letter in word:
            if letter not in pos.children:
                pos.children[letter] = Trie()
            pos = pos.children[letter]
        pos.word = word
            
            
    def find(self, word, allow_mistakes=0):
        
        # раскидываем возврат __find по разным сетам по числу ошибок
        result = [set() for i in range(allow_mistakes + 1)]
        suggestions = self.__find(word.lower(), allow_mistakes, 0)
        for word, errors in suggestions:  
            result[allow_mistakes - errors].add(word)
        
        for s in result:
            if len(s) > 0:
                return s  # set with the lowest # of errors
        return None
    
    
    # Возвращает набор {(слово, (допустимо ошибок - сделано ошибок))}
    def __find(self, word, errors_left, pos):        
        result = set()
                
        for child_letter, child_node in self.children.items():  # for every possible next letter
            if pos < len(word) and child_letter == word[pos]:
                result |= child_node.__find(word, errors_left, pos + 1)  # exact
            elif errors_left > 0:
                result |= child_node.__find(word, errors_left - 1, pos + 1)  # replace
                result |= child_node.__find(word, errors_left - 1, pos)  # insert  
        if errors_left > 0:
            result |= self.__find(word, errors_left - 1, pos + 1)  # delete
        
        if self.word is not None and abs(pos - len(word)) <= errors_left:  # delete in the end
            result.add((self.word, errors_left - abs(pos - len(word))))
        
        return result       

### Датасет

In [4]:
import re
from collections import Counter

def text_to_wordlist(sentence):
    regexp = "[^а-яА-Яё]"
    sentence = re.sub(regexp, " ", sentence)
    return sentence.lower().split()

def load_file(path):
    words = []
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = text_to_wordlist(text)
    return words

In [5]:
words = load_file("wp.txt")
print (words[:10])

['лев', 'николаевич', 'толстой', 'война', 'и', 'мир', 'том', 'часть', 'первая', 'е']


### Наполняем бор и пробуем искать

In [6]:
spell_checker = Trie()
for word in words:
    spell_checker.add(word)

In [9]:
test = ["лев", "ТоЛсТОй", "воина", "мир", "Наташа", "хайп", "банан"]
for t in test:
    print (t, spell_checker.find(t, allow_mistakes=2))

лев {'лев'}
ТоЛсТОй {'толстой'}
воина {'воин', 'война', 'волна', 'вина'}
мир {'мир'}
Наташа {'наташа'}
хайп {'нап', 'ха', 'тайн', 'ай', 'рай', 'дай', 'хаос', 'чай'}
банан {'рана', 'бала', 'барон', 'баланс', 'банке', 'барин', 'ранен', 'бантах', 'дана', 'канал', 'баба', 'балах', 'бани', 'ланн'}
